In [ ]:
!pip install vit_keras

In [ ]:
!pip install opencv-python

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout, Activation, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.datasets import cifar10
from vit_keras import vit, utils
import tensorflow_addons as tfa

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import gc
import numpy as np # linear algebra
import pandas as pd

/home/aivn48/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
# data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

X_train = (X_train/255.).astype("float32")
X_test  = (X_test/255.).astype("float32")

In [ ]:
batch_size = 128
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

In [ ]:
# model
input_shape = (32, 32, 3) # Cifar10 image size
image_size = 256 # size after resizing image
num_classes = 10

def build_model():
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (image_size, image_size)))(inputs)
    base_model = vit.vit_b16(image_size=image_size, pretrained=True,
                            include_top=False, pretrained_top=False)
    
    #base_model.trainable = False #Set false for transfer learning
    x = base_model(x)
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation=tfa.activations.gelu)(x)
    x = BatchNormalization()(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model_final = Model(inputs=inputs, outputs=outputs)
    return model_final

In [ ]:
model = build_model()
model.summary()

2023-01-02 15:01:06.941081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-02 15:01:06.958575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-02 15:01:06.958655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-02 15:01:06.959307: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 256, 256, 3)       0         
                                                                 
 vit-b16 (Functional)        (None, 768)               85844736  
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 batch_normalization (BatchN  (None, 768)              3072      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 32)                24608     
                                                             

In [ ]:
# train
model.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9), 
              loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_generator,
          epochs=15,
          validation_data=(X_test, y_test))

Epoch 1/15


2023-01-02 15:01:14.556529: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2023-01-02 15:01:15.289630: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


391/391 [==============================] - 547s 1s/step - loss: 0.2398 - accuracy: 0.9279 - val_loss: 0.0803 - val_accuracy: 0.9753
Epoch 2/15
391/391 [==============================] - 540s 1s/step - loss: 0.0992 - accuracy: 0.9684 - val_loss: 0.0752 - val_accuracy: 0.9782
Epoch 3/15
391/391 [==============================] - 540s 1s/step - loss: 0.0701 - accuracy: 0.9773 - val_loss: 0.0651 - val_accuracy: 0.9814
Epoch 4/15
391/391 [==============================] - 540s 1s/step - loss: 0.0570 - accuracy: 0.9819 - val_loss: 0.0477 - val_accuracy: 0.9846
Epoch 5/15
391/391 [==============================] - 541s 1s/step - loss: 0.0506 - accuracy: 0.9832 - val_loss: 0.0475 - val_accuracy: 0.9864
Epoch 6/15
391/391 [==============================] - 541s 1s/step - loss: 0.0407 - accuracy: 0.9870 - val_loss: 0.0493 - val_accuracy: 0.9851
Epoch 7/15
391/391 [==============================] - 540s 1s/step - loss: 0.0366 - accuracy: 0.9882 - val_loss: 0.0473 - val_accuracy: 0.9858
Epoch 8/15